In [ ]:
from transformers.pipelines import SUPPORTED_TASKS

In [ ]:
from pprint import pprint
pprint(SUPPORTED_TASKS.keys())

In [ ]:
for k, v in SUPPORTED_TASKS.items():
    print(k, v)

In [7]:
from transformers import pipeline, QuestionAnsweringPipeline

In [8]:
pipe = pipeline("text-classification")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /distilbert/distilbert-base-uncased-finetuned-sst-2-english/resolve/main/config.json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f64abc98cd0>: Failed to establish a new connection: [Errno 101] Network is unreachable'))"), '(Request ID: c2137247-5d13-41f1-a77a-6fc9a5bc3cc7)')' thrown while requesting HEAD https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english/resolve/main/config.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /distilbert/distilbert

KeyboardInterrupt: 